![title](https://image.ibb.co/erDntK/logo2018.png)

---




# Task 12 part 2/3 - Train CIFAR10 using GPU

In this second part of the assignment you will try to train a five layer CNN on CIFAR-10 dataset using GPU and feel the increased training speed compared to only using CPU

In this assignment you will also try to train the network using Data Augmentation


The goals of this assignment are as follows:

<pre>* train a five layer CNN using GPU from scratch
* train a five layer CNN using Data Augmentation 


---
---
#[Part 0] Import Libraries and Load Data

---
## 0 - Acceleration Setting

This time we will check the ability of TensorFlow if it is run with GPU acceleration.

For that, make sure that this Google Colab use **GPU** Runtime acceleration.
* Select the Runtime menu
* Change Runtime Type
* Choose **GPU**

<img src = "https://i.ibb.co/QX3Brf0/gpu.png" align = "center">



---
## 1 - Install TensorFlow 2

If Tensorflow 2 is not already installed, install it first

In [0]:
!pip install tensorflow-gpu -q

In [0]:
import tensorflow as tf
tf.__version__

**EXPECTED OUTPUT**:
<pre>
 '2.0.0'

In [0]:
import torch
torch.__version__

**EXPECTED OUTPUT**:
<pre>
 '1.3.0'

---
## 2 - Import Libraries
Import required libraries

In [0]:
import time
import numpy as np
import matplotlib.pyplot as plt
import os
import pprint

from tensorflow.keras import backend as K
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model

%matplotlib inline
np.set_printoptions(precision=7)
%load_ext autoreload
%autoreload 2

Write down your Name and Student ID

In [0]:
## --- start your code here ----

NIM = ??
Nama = ??

## --- end your code here ----

---
## 3 - Load CIFAR-10

In [0]:
(X_train_ori, y_train), (X_test_ori, y_test) = tf.keras.datasets.cifar10.load_data()

class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


---
## 4 - Split Validation Data

In [0]:
X_val_ori = X_train_ori[-10000:,:]
y_val     = y_train[-10000:]

X_train_ori = X_train_ori[:-10000, :]
y_train     = y_train[:-10000]

---
## 5 - Normalize and Reshape Data

In [0]:
X_train = X_train_ori.astype('float32')
X_val   = X_val_ori.astype('float32')
X_test  = X_test_ori.astype('float32')

mean_image = X_train.mean(axis=(0, 1, 2), keepdims=True)
std_image = X_train.std(axis=(0, 1, 2), keepdims=True)

X_train = (X_train - mean_image) /std_image
X_val = (X_val - mean_image) /std_image
X_test = (X_test - mean_image) /std_image

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')


print('X_train.shape =',X_train.shape)
print('X_val.shape   =',X_val.shape)
print('X_test.shape  =',X_test.shape)

y_train = y_train.ravel()
y_val   = y_val.ravel()
y_test  = y_test.ravel()

print('\ny_train.shape =',y_train.shape)
print('y_val.shape   =',y_val.shape)
print('y_test.shape  =',y_test.shape)

one hot the label

In [0]:
y_train_hot = to_categorical(y_train, 10)
y_val_hot   = to_categorical(y_val, 10)
y_test_hot  = to_categorical(y_test, 10)

print('y_train_hot.shape =',y_train_hot.shape)
print('y_val_hot.shape   =',y_val_hot.shape)
print('y_test_hot.shape  =',y_test_hot.shape)

---
## 6 - Helper Function

In [0]:
def plot_history(history):
  plt.rcParams['figure.figsize'] = [12, 4]
  plt.subplots_adjust(wspace=0.2)

  plt.subplot(121)
  # Plot training & validation accuracy values
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'])

  plt.subplot(122)
  # Plot training & validation loss values
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Val'])
  plt.show()

---
---
# [Part 1] Device Acceleration Checking

## 1 - GPU Availability Check

Next, let's check TensorFlow and PyTorch's ability to utilize GPU capabilities

**Because we set this Google Colab to run with GPU acceleration, Tensorflow will return the GPU device id**

**If we're using PyTorch to check the deivice, we can see the GPU name**

In [0]:
print('Using GPU:', tf.test.is_gpu_available())

if tf.test.is_gpu_available():
  print('GPU name :', tf.test.gpu_device_name())


**EXPECTED OUTPUT**:
<pre>
    Using GPU: True
    GPU name : /device:GPU:0

In [0]:
print('Using GPU:',torch.cuda.is_available())

if torch.cuda.is_available():
  print('GPU name :',torch.cuda.get_device_name(0))


**EXPECTED OUTPUT**:
<pre>
    Using GPU: True
    GPU name : Tesla K80

## 2 - TPU Availability Check

Then, let's check TensorFlow's ability to utilize TPU capabilities 

**Because we set this Google Colab to run with GPU acceleration, there will be no TPU acceleration that can be used**

In [0]:
if 'COLAB_TPU_ADDR' not in os.environ:
  print('Not connected to a TPU runtime')
  
else:
  
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  # for tf 2.x
  # tf.config.experimental_connect_to_host(tpu_address)
  # devices=tf.config.experimental_list_devices()
  
  # for tf 1.x  
  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:') 
  print(*devices, sep="\n")

**EXPECTED OUTPUT**:
<pre>
Not connected to a TPU runtime

---
---
# [Part 2] Train 5-layer ConvNet

Now let's build and train our model



In [0]:
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, GlobalAveragePooling2D


---
## 1 - Model Builder

#### <font color='red'>**EXERCISE**: </font>
**Define a ConvNet Model as follow**

    * 32 @ Conv 3x3 using relu and padding same
    * 32 @ Conv 3x3 using relu and padding same
    * Max Pool
    * 64 @ Conv 3x3 using relu and padding same    
    * Max Pool
    * Flatten
    * Dense 200 using relu
    * Dense 10 using softmax
    
    then compile using categorical crossentropy and optimizer adam


In [0]:
def FiveConvNet(name):
  
  model = Sequential([

      ??, # conv
      ??, # conv
      ??, # pool
      ??, # conv
      ??, # pool
      ??, # flatten
      ??, # dense
      ??  # dense
      
  ], name = name) 
  
  model.compile(??, ??, metrics=['accuracy'])
  
  return model

---
## 2 - Define Model

In [0]:
model = FiveConvNet('using_GPU')

model.summary()

**EXPECTED OUTPUT**:
<pre>
Model: "using_GPU"
conv2d (Conv2D)              (None, 32, 32, 32)        896       
...
dense_? (Dense)              (None, 10)                2010        

Total params: 850,050
Trainable params: 850,050
Non-trainable params: 0
_________________________________________________________________

---
## 3 - Train Model



#### <font color='red'>**EXERCISE**: </font>
**Train the model for 8 epochs using batch size 200**

**Don't forget to feed the validation data**


    

In [0]:
import time

num_epochs = ??
batch_size = ??

tic = time.time()

history = model.fit(??, ??, 
                    ??,
                    ??, 
                    ??, 
                    verbose=2)

toc = time.time()

print('\n\ntraining speed = %.2f seconds' % (toc-tic))
print('training speed = %.2f minutes' % ((toc-tic)/60))

**EXPECTED OUTPUT**:
<pre>
Your model should run in about 5 seconds per epoch
it should be about 30x faster than using CPU

---
## 4 - Visualize Training

In [0]:
plot_history(history)

---
## 5 - Evaluate Model

In [0]:
val_scores   = model.evaluate(X_val, y_val_hot, verbose=2)
test_scores  = model.evaluate(X_test, y_test_hot, verbose=2)

**EXPECTED OUTPUT**:
<pre>
You should get around 72% accuracy on data test
tough the model is overfit

---
---
# [Part 3] Train ConvNet using Data Augmentation

In this part, we'll use the same 5 layer convnet model, but let's train it using Data Augmentation

We'll use the `ImageDataGenerator` function provided by `tf.keras`



---
## 1 - Define Model

Initiate new model

In [0]:
model_aug = FiveConvNet('Augmented')

model_aug.summary()

---
## 2 - Define Augmentation

Keras already provided a set of function to automatically perform Data Augmentation on the run while we train the data. This way, we don't need to manually  modify and store the data

you can read more detailed example [here](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

The first step is by defining what type of Augmentation we want to perform to our data

#### <font color='red'>**EXERCISE**: </font>
**Define your augmentation scheme**

    Common setting:
    * width and height shift 0.1 to 0.2
    * horizontal flip true
    * vertical flip false
    * zoom 0.1 to 0.2
    * rotation


    

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


datagen = ImageDataGenerator(
    
    rotation_range=??,
    
    width_shift_range=??,
    
    height_shift_range=??,
    
    brightness_range=[??, ??],
    
    shear_range=??,
    
    zoom_range=??,
    
    channel_shift_range=??,
    
    horizontal_flip=??,
    
    vertical_flip=??,
    
    rescale=??,
    
)


Prepare the data generator to fit the train data

In [0]:
datagen.fit(X_train)

---
## 3 - Train Model

To train the model using Data Augmentation, we're not going to use `.fit()` function, 

but instead, we use `.fit_generator()` function

when passing the data train to the `.fit_generator()`  function, we'll use `datagen.flow()` to perform the augmentation on the run

the use is as follow

```python
augmented = datagen.flow(data, target, batch_size=??)

history = model.fit_generator( augmented, validation=??, epoch=??)

```

#### <font color='red'>**EXERCISE**: </font>
**Train the model for 8 epochs using batch size 200**

**Don't forget to feed the validation data**


    

In [0]:
import time

num_epochs = ??
batch_size = ??

tic = time.time()

# call datagen.flow() function with input X_train, y_train_hot, and batch_size
augmented_train = datagen.flow(
    ??, ??, ??
)


# call model_aug.fit_generator() function with input augmented_train,
# validation_data = (X_val, y_val_hot), and epoch
history_aug = model_aug.fit_generator(
    ??,
    ??
    ??, 
    verbose=2)

toc = time.time()

print('\n\ntraining speed = %.2f seconds' % (toc-tic))
print('training speed = %.2f minutes' % ((toc-tic)/60))

**EXPECTED OUTPUT**:
<pre>
Your model should run in about 30-50 seconds per epoch
depending on your Augmentation scheme

---
## 4 - Visualize Training

In [0]:
plot_history(history_aug)

---
## 5 - Evaluate Model

In [0]:
val_scores   = model_aug.evaluate(X_val, y_val_hot, verbose=2)
test_scores  = model_aug.evaluate(X_test, y_test_hot, verbose=2)

**EXPECTED OUTPUT**:
<pre>
You should get around 76% accuracy on data test

You should also see that using Data Augmentation prevent Overfitting

---
---
# [Part 4] Train CIFAR10 using VGG

Our 5-layer ConvNet already runs fast with 5 seconds per epoch

Not much from here that can be accelerated anymore

So instead, let's define a bigger model using VGG16 so we can later compare it with TPU accelerated computation



---
## 1 - Model Builder

Load empty VGG16, cut it up to the 4th block, then add a Global Average Pooling and a couple of Dense Layer

#### <font color='red'>**EXERCISE**: </font>
**Define a ConvNet Model as follow**

    * vgg16 model with weights=None, include_top=False, and input_shape=(32,32,3)
    * get layer 'block4_conv3' output
    * GlobalAveragePooling2D
    * Dense 200 using relu
    * Dense 10 using softmax
    
    use Functional API,
    then compile using categorical crossentropy and optimizer adam


In [0]:
from tensorflow.keras.applications.vgg16 import VGG16

def CifarVGG(name):
  
  # load empty vgg16 no top with input (32,32,3)
  model = VGG16(weights=None, include_top=False, input_shape=(32, 32, 3))

  # get 'block4_conv3' output
  x = model.get_layer('block4_conv3').output
  
  # global average pooling
  x = ??

  # dense 200
  x = ??

  # dense 10
  prediction = ??
  
  # instantiate model
  myModel = Model(inputs=model.input, outputs=prediction, name=name)

  myModel.compile(??,  ??, metrics=['accuracy'])
  
  return myModel
  

---
## 2 - Define Model

In [0]:
model = CifarVGG('VGG_using_GPU')

model.summary()

**EXPECTED OUTPUT**:
<pre>
Model: "VGG_using_GPU"
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
...
dense_? (Dense)              (None, 200)               102600    
prediction (Dense)           (None, 10)                2010      

Total params: 7,739,874
Trainable params: 7,739,874
Non-trainable params: 0
_________________________________________________________________

---
## 3 - Train Model

Now train the model for 10 epoch

For greater comparison, we'll use batch size of 1000

But this time, notice that we're not passing validation set


In [0]:
import time

num_epochs = 10
batch_size = 1000

tic = time.time()

history = model.fit(X_train, y_train_hot, 
                    epochs=num_epochs, 
                    batch_size=batch_size, 
                    verbose=2)

toc = time.time()

print('\n\ntraining speed = %.2f seconds' % (toc-tic))
print('training speed = %.2f minutes' % ((toc-tic)/60))

**EXPECTED OUTPUT**:
<pre>
Your model should run in about 20 seconds per epoch

---
## 4 - Evaluate Model

In [0]:
val_scores   = model.evaluate(X_val, y_val_hot, verbose=2)
test_scores  = model.evaluate(X_test, y_test_hot, verbose=2)

**EXPECTED OUTPUT**:
<pre>
You should get around 70% accuracy on data test



---

# Congratulation, You've Completed Exercise 12 part 2/3

<p>Copyright &copy;  <a href=https://www.linkedin.com/in/andityaarifianto/>2019 - ADF</a> </p>

![footer](https://image.ibb.co/hAHDYK/footer2018.png)